## Examples

In [1]:
from thecodecrate_pipeline import Pipeline

pipeline = Pipeline()

In [2]:
# simple usage
from thecodecrate_pipeline import (
    Pipeline,
)

pipeline = (
    (Pipeline[int, str]())
    .pipe(lambda x: x + 1)
    .pipe(lambda x: x + 1)
    .pipe(lambda x: f"result is {x}")
)

# 'result is 3'
await pipeline.process(1)

'result is 3'

In [3]:
# with class-based stages
from thecodecrate_pipeline import (
    StageInterface,
)


class TimesTwoStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload * 2


class AddOneStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload + 1


my_pipeline = (Pipeline[int]()).pipe(TimesTwoStage()).pipe(AddOneStage())

# returns 21
result = await my_pipeline.process(10)
assert result == 21
result

21

In [4]:
# with pipelines as stages
class TimesThreeStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload * 3


class AddFiveStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload + 5


pipeline = (
    (Pipeline[int]()).pipe(TimesThreeStage()).pipe(my_pipeline).pipe(AddFiveStage())
)

# returns 36
# 5 * 3 = 15 * 2 + 1 = 31 + 5 = 36
result = await pipeline.process(5)
assert result == 36
result

36

In [5]:
# with stages declared on the class
class MyPipeline(Pipeline[int]):
    stages = (
        TimesThreeStage(),
        my_pipeline,
        AddFiveStage,
    )


# returns 36
result = await MyPipeline().process(5)
assert result == 36
result

36

In [6]:
###
## Stages with Custom Arguments
##
## On this example, the stages have the current index
## of the processing queue
###
from abc import abstractmethod
from typing import Awaitable, Callable, Concatenate, cast

from thecodecrate_pipeline import (
    Processor,
    Stage,
    T_in,
    T_out,
    StageInstanceCollection,
)


class IndexedStage(Stage[T_in, T_out]):
    @abstractmethod
    async def __call__(
        self,
        payload: T_in,
        /,
        tag: int,
    ) -> T_out:
        pass


IndexedPipelineCallable = (
    IndexedStage[T_in, T_out]
    | Callable[Concatenate[T_in, ...], Awaitable[T_out]]
    | Callable[Concatenate[T_in, ...], T_out]
)


class IndexedProcessor(Processor[T_in, T_out]):
    async def process(
        self,
        payload: T_in,
        stages: StageInstanceCollection,
    ) -> T_out:
        index = 0

        for stage in stages:
            payload = await self._call(
                callable=stage,
                payload=payload,
                index=index,
            )

            index += 1

        return cast(T_out, payload)


class IndexedPipeline(Pipeline[T_in, T_out]):
    processor_class = IndexedProcessor


###
## Usage
###
class MyIndexedStage(IndexedStage[str]):
    # type-hinting: change `index` type to see an error
    async def __call__(self, payload: str, index: int) -> str:
        return f"{payload}: {index}"


indexed_pipeline = (
    (IndexedPipeline[str]()).pipe(MyIndexedStage()).pipe(MyIndexedStage())
)

assert await indexed_pipeline.process("test") == "test: 0: 1"

# returns "hello: 0: 1"
result = await indexed_pipeline.process("hello")
assert result == "hello: 0: 1"
result

'hello: 0: 1'

In [7]:
from thecodecrate_pipeline import InterruptiblePipeline


def continues_when_less_than_ten(payload: int) -> bool:
    return payload < 10


pipeline = (
    InterruptiblePipeline[int](continues_when_less_than_ten)
    .pipe(lambda payload: payload + 1)
    .pipe(lambda payload: payload * 2)
    .pipe(lambda payload: payload * 3)
)

# returns 12
result = await pipeline.process(5)
assert result == 12
result

12